В этом файле собирается и обучается модель 

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_splitВ
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
import joblib
import os

# Путь к папке с сохранёнными файлами
load_dir = 'PythonProjectMarina'

# Загрузка переменных
X_train_base = joblib.load(os.path.join(load_dir, 'X_train_base.pkl'))
X_val_base = joblib.load(os.path.join(load_dir, 'X_val_base.pkl'))
X_train_comparison = joblib.load(os.path.join(load_dir, 'X_train_comparison.pkl'))
X_val_comparison = joblib.load(os.path.join(load_dir, 'X_val_comparison.pkl'))
y_train = joblib.load(os.path.join(load_dir, 'y_train.pkl'))
y_val = joblib.load(os.path.join(load_dir, 'y_val.pkl'))


In [ ]:
from tensorflow.keras import layers, models, optimizers, Model

input_shape = (220, 155, 1)  

x1 = layers.Input(shape=input_shape)
x2 = layers.Input(shape=input_shape)

def build_feature_extractor(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(128, 3, activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    return Model(inputs=inputs, outputs=x)

feature_model = build_feature_extractor(input_shape)

x1_feat = feature_model(x1)
x2_feat = feature_model(x2)

net = layers.Subtract()([x1_feat, x2_feat])
net = layers.Dense(64, activation='relu')(net)
net = layers.Dense(1, activation='sigmoid')(net)

classifier = Model(inputs=[x1, x2], outputs=net)
classifier.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['accuracy']
)


In [ ]:
history = classifier.fit(
    [X_train_base, X_train_comparison], y_train, 
    epochs=5,
    batch_size=32,
    validation_data=([X_val_base, X_val_comparison], y_val)
)

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Loss per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Accuracy per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
classifier.save('my_model.keras')